In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [76]:
# input
phe_info_f <- '../../common/biomarker.phenotype.info.tsv'
phe_raw_f <- '/oak/stanford/groups/mrivas/projects/biomarkers/covariate_corrected/outputExtendedNoTDIreduced/phenotypes/full.table.phe'
phe_raw_add1_f <- '/oak/stanford/groups/mrivas/projects/biomarkers/covariate_corrected/outputExtendedNoTDIreduced/phenotypes/derived/AST_ALT_ratio.phe'
phe_raw_add2_f <- '/oak/stanford/groups/mrivas/projects/biomarkers/covariate_corrected/outputExtendedNoTDIreduced/phenotypes/full.table.glucose.phe'
phe_residual_f <- '/oak/stanford/groups/mrivas/projects/biomarkers/covariate_corrected/outputExtendedNoTDIreduced/phenotypes/combined.20190810.phe'
pop_def_f <- '/oak/stanford/groups/mrivas/ukbb24983/sqc/population_stratification_w24983_20190809/ukb24983_GWAS_covar.20190809.phe'
split_f <- '/oak/stanford/groups/mrivas/ukbb24983/sqc/population_stratification_w24983_20190809/split/ukb24983_white_british_%s.phe'
prs_f <- '/oak/stanford/groups/mrivas/projects/biomarkers/snpnet/biomarkers/%s/results/score/%s.sscore'

# output
out_long_phe_f <- '/oak/stanford/groups/mrivas/projects/biomarkers/snpnet/biomarkers/biomarkers.eval.long.tsv'
out_f <- 'snpnet_prs_eval.tsv'


## population definition, train/val/split split

In [31]:
pop_def_df <- fread(
    pop_def_f, select=c('FID', 'IID', 'population'),
    colClasses=c('FID'='character', 'IID'='character')
)


In [49]:
split_df <- c('train', 'val', 'test') %>%
lapply(function(s){
    fread(sprintf(split_f, s), colClasses='character', col.names=c('FID', 'IID')) %>%
    mutate(split = s)
}) %>% 
bind_rows() %>%
filter(FID %in% (pop_def_df %>% filter(population == 'white_british') %>% pull(FID)))


In [56]:
pop_split_df <- pop_def_df %>%
left_join(split_df, by=c('FID', 'IID')) %>%
mutate(
    pop_split = if_else(is.na(split), population, paste(population, split, sep=':'))
) %>%
arrange(as.numeric(FID))

In [57]:
pop_split_df %>% count(pop_split)


pop_split,n
<chr>,<int>
african,6498
e_asian,1154
non_british_white,24909
s_asian,7885
white_british:test,67430
white_british:train,236005
white_british:val,33716
NA,110780


## phenotype
### phenotype info

In [8]:
phe_info_df <- fread(phe_info_f) %>% rename('Phenotype'='name')

In [10]:
phe_info_df %>% pull(annotation)

[1] "Alanine_aminotransferase"    "Albumin"                    
 [3] "Alkaline_phosphatase"        "Apolipoprotein_A"           
 [5] "Apolipoprotein_B_adjstatins" "Aspartate_aminotransferase" 
 [7] "AST_ALT_ratio"               "C_reactive_protein"         
 [9] "Calcium"                     "Cholesterol_adjstatins"     
[11] "Creatinine"                  "Creatinine_in_urine"        
[13] "Cystatin_C"                  "Direct_bilirubin"           
[15] "eGFR"                        "Gamma_glutamyltransferase"  
[17] "Glucose"                     "Glycated_haemoglobin_HbA1c" 
[19] "HDL_cholesterol"             "IGF_1"                      
[21] "LDL_direct_adjstatins"       "Lipoprotein_A"              
[23] "Microalbumin_in_urine"       "Non_albumin_protein"        
[25] "Phosphate"                   "Potassium_in_urine"         
[27] "SHBG"                        "Sodium_in_urine"            
[29] "Testosterone"                "Total_bilirubin"            
[31] "Total_protein"               "Triglycerides"              
[33] "Urate"                       "Urea"                       
[35] "Vitamin_D"

### raw phenotype

In [23]:
phe_raw_df <- fread(
    phe_raw_f, 
    select=c('FID', 'IID', phe_info_df %>% pull(annotation)), 
    colClasses=c('FID'='character', 'IID'='character')
) %>%
select(-Glucose) %>%
left_join(
    fread(
        phe_raw_add1_f, select=c('FID', 'IID', 'AST_ALT_ratio'),
        colClasses=c('FID'='character', 'IID'='character')  
    ), by=c('FID', 'IID')
) %>%
left_join(
    fread(
        phe_raw_add2_f, select=c('FID', 'IID', 'Glucose'), 
        colClasses=c('FID'='character', 'IID'='character')        
    ), by=c('FID', 'IID')
)


Warning message in fread(phe_raw_f, select = c("FID", "IID", phe_info_df %>% pull(annotation)), :
“Column name 'AST_ALT_ratio' not found in column name header (case sensitive), skipping.”

### residual

In [26]:
phe_residual_df <- fread(
    phe_residual_f, 
    select=c('FID', 'IID', phe_info_df %>% pull(annotation)), 
    colClasses=c('FID'='character', 'IID'='character')
)


In [27]:
phe_raw_df %>% dim()

[1] 461503     37

In [28]:
phe_residual_df %>% dim()

[1] 461503     37

## PRS

In [45]:
sscore_df <- phe_info_df %>% pull(annotation) %>%
lapply(function(phe){
    fread(
        sprintf(prs_f, phe, phe),
        select=c('#FID', 'IID', 'SCORE1_SUM'),
        colClasses=c('#FID'='character', 'IID'='character')
    ) %>%
    rename('FID'='#FID') %>%
    rename(!! phe := 'SCORE1_SUM')
}) %>%
reduce(function(x, y) inner_join(x, y, by=c('FID', 'IID')))


In [46]:
sscore_df %>% dim()

[1] 488377     37

## join

In [69]:
long_df <- sscore_df %>%
gather(trait, PRS, -FID, -IID) %>%
full_join(
    phe_raw_df %>%
    gather(trait, raw_phe, -FID, -IID),
    by=c('FID', 'IID', 'trait')
) %>%
full_join(
    phe_residual_df %>%
    gather(trait, residual_phe, -FID, -IID),
    by=c('FID', 'IID', 'trait')
) %>%
left_join(
    pop_split_df %>%
    select(FID, IID, pop_split),
    by=c('FID', 'IID')
) %>%
mutate(
    covar_score = raw_phe - residual_phe,
    total_score = covar_score + PRS
)


In [74]:
long_df %>%
rename('#FID' = 'FID') %>%
fwrite(out_long_phe_f, sep='\t', na = "NA", quote=F)


In [77]:
out_long_phe_f

[1] "/oak/stanford/groups/mrivas/projects/biomarkers/snpnet/biomarkers/biomarkers.eval.long.tsv"

## eval

In [122]:
r2 <- function(response, pred){
#     1 - sum((response - pred)^2) / sum((response - mean(response))^2)    
    summary(lm(response ~ 1 + pred))$r.squared
}


In [123]:
build_eval_df <- function(long_df, phe, p_s){
    df <- long_df %>% 
    filter(trait == phe, pop_split == p_s) %>%
    drop_na(raw_phe, PRS)
    
    data.frame(
        trait      = phe,
        pop_split  = p_s,
        geno       = r2(df$raw_phe, df$PRS),
        covar      = r2(df$raw_phe, df$covar_score),
        geno_covar = r2(df$raw_phe, df$total_score),
        stringsAsFactors = F
    )    
}


In [132]:
# test with example
build_eval_df(long_df, phe = 'Testosterone', p_s = 'white_british:train')

trait,pop_split,geno,covar,geno_covar
<chr>,<chr>,<dbl>,<dbl>,<dbl>
Testosterone,white_british:train,0.01268084,0.9958828,0.9963106


In [129]:
r2_eval_df <- pop_split_df %>%
drop_na(pop_split) %>%
pull(pop_split) %>%
unique() %>%
lapply(function(p_s){
    phe_info_df %>% pull(annotation) %>%
    lapply(function(phe){
        build_eval_df(long_df, phe, p_s)
    }) %>% bind_rows()
}) %>% bind_rows()


In [131]:
r2_eval_df %>% dim()

[1] 245   5

In [133]:
r2_eval_df %>%
rename('#trait' = 'trait') %>%
fwrite(out_f, sep='\t', na = "NA", quote=F)


In [134]:
out_f

[1] "snpnet_prs_eval.tsv"